# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.core import Workspace, Experiment, RunConfiguration
from azureml.train.sklearn import SKLearn

from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core.model import Model

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import os

import requests
import json

import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-heart-stroke'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# Create compute cluster
compute_cluster_name = "hyper-drive-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [41]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.01, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"C": uniform(0.1, 1.0), 
                                 "max_iter": choice(50,100,150,200,250)})
env = Environment.from_conda_specification(name='training_environment', file_path='./conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
script_run_config = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=env
                      )

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name="Accuracy", 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=script_run_config,
                                     max_total_runs=10, 
                                    max_concurrent_runs=5)

In [42]:
experiment = Experiment(ws, experiment_name)
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [43]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_095c43f4-d76b-4172-99dd-f751a965dc2d
Web View: https://ml.azure.com/runs/HD_095c43f4-d76b-4172-99dd-f751a965dc2d?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-262077/workspaces/quick-starts-ws-262077&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-07-09T18:08:08.805924][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2024-07-09T18:08:09.4158950Z][SCHEDULER][INFO]Scheduling job, id='HD_095c43f4-d76b-4172-99dd-f751a965dc2d_0' 
[2024-07-09T18:08:09.5496166Z][SCHEDULER][INFO]Scheduling job, id='HD_095c43f4-d76b-4172-99dd-f751a965dc2d_1' 
[2024-07-09T18:08:09.6436959Z][SCHEDULER][INFO]Scheduling job, id='HD_095c43f4-d76b-4172-99dd-f751a965dc2d_2' 
[2024-07-09T18:08:09.7555510Z][SCHEDULER][INFO]Scheduling job, id='HD_095c43f4-d76b-4172-99dd-f751a965dc2d_3' 
[2024-07-09T18:08:09.8724717Z][SCHEDULER][INFO]Scheduling job, id='HD_095c43f4-d76b-4172-99dd-f751a965dc2d_4' 
[2024-07-09

{'runId': 'HD_095c43f4-d76b-4172-99dd-f751a965dc2d',
 'target': 'hyper-drive-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-07-09T18:08:07.665736Z',
 'endTimeUtc': '2024-07-09T18:16:42.251746Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8c82cfa4-25bf-412c-87e9-dd707f6f5e9b',
  'user_agent': 'python/3.9.19 (Linux-5.15.0-1064-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_095c43f4-d76b-4172-99dd-f751a965dc2d_8',
  'score': '0.9517286366601436',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_095c43f4-d76b-4172-99dd-f751a965dc2d_8'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'tele

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [44]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run)

best_run_metrics = best_hyperdrive_run.get_metrics()
print(best_run_metrics)

print('RunID: ', best_hyperdrive_run.id)

Run(Experiment: hyperdrive-heart-stroke,
Id: HD_095c43f4-d76b-4172-99dd-f751a965dc2d_8,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 0.9972268900280543, 'Max iterations:': 50, 'Accuracy': 0.9517286366601435}
RunID:  HD_095c43f4-d76b-4172-99dd-f751a965dc2d_8


In [45]:
print(best_hyperdrive_run.get_details()['runId'])


HD_095c43f4-d76b-4172-99dd-f751a965dc2d_8


In [46]:
print(best_hyperdrive_run.get_file_names())


['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2024-07-09-18', 'outputs/hyperdrive_best_run.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [107]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

# Create environment from the YAML file
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create inference configuration
inference_config = InferenceConfig(entry_script='score.py', environment=sklearn_env)

# Define deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(ws, 'stroke-1', [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

print(service.state)
print(service.scoring_uri)


azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-07-09 19:59:09+00:00 Creating Container Registry if not exists.
2024-07-09 19:59:10+00:00 Registering the environment.
2024-07-09 19:59:12+00:00 Use the existing image.
2024-07-09 19:59:12+00:00 Generating deployment configuration.
2024-07-09 19:59:13+00:00 Submitting deployment to compute..
2024-07-09 19:59:26+00:00 Checking the status of deployment stroke-1..
2024-07-09 20:01:36+00:00 Checking the status of inference endpoint stroke-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://c67b1044-dd56-4f17-b288-6fa7c75c6601.westeurope.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [109]:
import urllib.request
import json
import ssl

def allowSelfSignedHttps(allowed):
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

data = {"data": [[0, 0, 0, 0, 0, False, 0, 0, 0, 0, 0]]}
body = str.encode(json.dumps(data))

url = 'http://c67b1044-dd56-4f17-b288-6fa7c75c6601.westeurope.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"predictions\\": [0]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

